In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

In [ ]:
import os
# Specify the new directory path
parent_directory = 'c:/Zhou/Projects/Corn-Yield'

# Change the current directory
os.chdir(parent_directory) 
# %pwd

In [ ]:
import numpy as np

In [ ]:
from dataset import select_data_and_yield_list, get_ml_image
from dl.dl_dataset import CornDataset

In [ ]:
transform = transforms.Compose([
    transforms.Resize((80, 80)),  # Resize the images to a fixed size
    transforms.ToTensor()           # Convert images to tensors
    # transforms.Normalize(mean=[0.04573556, 0.08850508, 0.10431199, 0.19097967, 0.34651175], 
    #                      std=[0.01902218, 0.02884705, 0.05737383, 0.0515388, 0.0798206 ])  # Normalize pixel values
])

In [ ]:
yield_file = 'C:/Users/yutzhou/Desktop/Corn_Yield/BL2022_Yld.csv'
img_path = 'C:/Users/yutzhou/Desktop/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220926_DOY269_extracted_filled'
out_path = 'output/'

name_tag = img_path[-23:-17]
# print(name_tag)

# key_word_list = ['Ref_filled.tif', 'RGB_filled.tif']
key_word_list = ['Ref_filled.tif']
suffix_list_list = [[], ['LWIR_filled.tif']]

# suffix = ['base', 'lwir']
# suffix_list = ['LWIR_filled.tif']
# VI_list = ['ndvi', 'ndre', 'gndvi', 'evi']
VI_list = ['evi']
for keyword in key_word_list:

    selection = ['Pioneer'] # 
    # selection = 'Pioneer Deficit' 
    # selection = 'Pioneer Full'
    pioneer_img_list, pioneer_yield_list, irrigate_type_list = select_data_and_yield_list(
        img_path, yield_file, key_word = keyword, crop_type_select=selection)
    
    
    # for suffix_list in suffix_list_list:
    
    # all_dataset = CornDataset(pioneer_img_list, pioneer_yield_list, transform = transform)
    all_dataset = CornDataset(pioneer_img_list, pioneer_yield_list)
    
    total_size = len(all_dataset)
    train_size = int(0.8 * total_size)  # 70% for training
    test_size = int(0.2 * total_size)   # 15% for validation
    
    g = torch.Generator()
    g.manual_seed(39)
    
    train_val_dataset, test_dataset = random_split(all_dataset, [train_size, test_size])


In [ ]:
print(type(train_val_dataset))

In [ ]:

# pioneer_dataset = get_ml_image(pioneer_img_list)
    
# MEAN = np.nanmean(pioneer_dataset,(0,2,3))
# STD = np.nanstd(pioneer_dataset, (0,2,3))
# print(MEAN, STD)

In [ ]:
import torch.nn.functional as F
import torch.nn as nn
class myNet(nn.Module):
    def __init__(self, in_channel):
        super().__init__()
        #self.conv1 = nn.Conv2d(in_channel, kernel_size=7, stride=2, padding=3, bias=False)
        self.conv1 = nn.Conv2d(in_channel,64, kernel_size=(5,7), stride=1, padding=(2,3), bias=False)
        self.conv2 = nn.Conv2d(64,128, kernel_size=(3,7), stride=1, padding=0, bias=False)
        self.conv3 = nn.Conv2d(128,256, kernel_size=(3,5), stride=1, padding=0, bias=False)
        self.pool = nn.MaxPool2d(2, 2)
        self.bn2 = nn.BatchNorm2d(128)
        self.bn3 = nn.BatchNorm2d(256)
        self.fc1 = nn.Linear(256*37*3, 4098)
        self.fc2 = nn.Linear(4098, 1028)
        self.fc3 = nn.Linear(1028, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = self.bn3(F.relu(self.conv3(x)))
        # print(x.size(1),  x.size(2), x.size(3))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        
        # print(x.size())
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [30]:
from sklearn.model_selection import KFold
import torch.optim as optim
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=39)

in_channel = 5
num_epochs = 5
batch_size=32

# Initialize an empty list to store fold-wise performance
fold_accuracies = []

# Initialize a new model for each fold
model = myNet(in_channel)

criterion = nn.MSELoss()  # Mean Squared Error loss function
# optimizer = optim.Adam(list(conv.parameters()) + list(deconv.parameters()), lr=0.001)  # Adam optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

# optimizer.zero_grad()
            
for train_indices, val_indices in kf.split(train_val_dataset):
    train_dataset = torch.utils.data.Subset(train_val_dataset, train_indices)
    val_dataset = torch.utils.data.Subset(train_val_dataset, val_indices)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)


    # Train your model on the training data
    for epoch in range(num_epochs):
        for i, data in enumerate(train_loader, 0):
            # Get inputs and labels
            inputs, labels = data
            # print(labels.shape)
            labels = labels.unsqueeze(1) #change to (32,1)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            
            # Forward pass
            outputs = model(inputs)
            # print(outputs)
            # print(labels)
            # break
            try :
            # Compute loss
            loss = criterion(outputs, labels)
            
            # Backward pass and optimization
            loss.backward()
            # update weights
            optimizer.step()
            
            # # Print statistics
            # running_loss += loss.item()
            # train_loss +=  loss.item()
            # if i % 100 == 99:    # Print every 100 mini-batches
            #     print('[%d, %5d] loss: %.3f' %
            #         (epoch + 1, i + 1, running_loss))
            #     running_loss = 0.0


    # Evaluate the model on the validation data
    model.eval()
    val_accuracy = []
    for i, data in enumerate(val_loader, 0):
        inputs, labels = data

        # Perform forward and backward passes through the neural network
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        val_accuracy.append(loss.detach().numpy())
        
        # Evaluation loop
    val_accuracy = np.mean(val_accuracy)
    fold_accuracies.append(val_accuracy)

# Calculate the average accuracy across all folds
average_accuracy = np.mean(fold_accuracies)

torch.Size([32])
torch.Size([32, 1])
torch.Size([32])
torch.Size([32, 1])
torch.Size([32])
torch.Size([32, 1])
torch.Size([32])
torch.Size([32, 1])
torch.Size([32])
torch.Size([32, 1])
torch.Size([32])
torch.Size([32, 1])
torch.Size([32])
torch.Size([32, 1])
torch.Size([32])
torch.Size([32, 1])
torch.Size([32])
torch.Size([32, 1])
torch.Size([32])
torch.Size([32, 1])
torch.Size([32])
torch.Size([32, 1])
torch.Size([32])
torch.Size([32, 1])
torch.Size([32])
torch.Size([32, 1])
torch.Size([32])
torch.Size([32, 1])
torch.Size([32])
torch.Size([32, 1])
torch.Size([32])
torch.Size([32, 1])
torch.Size([32])
torch.Size([32, 1])
torch.Size([32])
torch.Size([32, 1])
torch.Size([32])
torch.Size([32, 1])
torch.Size([32])
torch.Size([32, 1])
torch.Size([32])
torch.Size([32, 1])
torch.Size([32])
torch.Size([32, 1])
torch.Size([32])
torch.Size([32, 1])
torch.Size([32])
torch.Size([32, 1])
torch.Size([32])
torch.Size([32, 1])
torch.Size([32])
torch.Size([32, 1])
torch.Size([32])
torch.Size([32, 1])
t

In [ ]:
print(outputs)
print(labels)

In [ ]:
print(np.sqrt(average_accuracy))

In [ ]:
batch_size = 32  # Adjust the batch size as needed
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
train_dataset = 

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)